In [0]:
%run "./config/notebook_config"

In [0]:
dbutils.widgets.text("game_name", "example game", "Game Name to Delete")


In [0]:
# Set catalog
spark.sql(f"USE CATALOG {catalog_name}")

In [0]:
game_name = dbutils.widgets.get("game_name")
tables = [t.name for t in spark.catalog.listTables(database_name)]
print("Tables to delete: " + str(tables))


In [0]:
def count_rows_by_game(table_list, game_name):
    # Create a dictionary with the row counts corresponding to
    # game_name in each table.
    results = {}
    for table in table_list:
        df = spark.table(f"{catalog_name}.{database_name}.{table}")
        count = df.filter(df["game_name"] == game_name).count()
        results[table] = count
    return results

In [0]:
# See what current counts are for the given game in each table
counts = count_rows_by_game(tables, game_name)
print(game_name)
for table, count in counts.items():
    print(f"{table}: {count}")

In [0]:
#execute delete for each table
#{catalog_name}.{database_name}.{table}
for table in tables:
    spark.sql(
        f"""
        DELETE FROM {catalog_name}.{database_name}.{table}
        WHERE game_name = "{game_name}"
        """
    )

In [0]:
# confirm deletion with record count
counts = count_rows_by_game(tables, game_name)
print(game_name)
for table, count in counts.items():
    print(f"{table}: {count}")